# Image Quilting

In [108]:
# Imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import convolve2d

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [109]:
def find_random_patch(sample, patchsize, x_range, y_range):
    x = np.random.randint(x_range[0], x_range[1])
    y = np.random.randint(y_range[0], y_range[1])
    half_size = patchsize // 2
    patch = sample[y - half_size:y + half_size + 1, x - half_size:x + half_size + 1, :]

    if patch.shape[0] != patchsize or patch.shape[1] != patchsize:
        raise ValueError(f"Patch size mismatch: Expected ({patchsize}, {patchsize}), got {patch.shape[:2]}")

    return patch

def choose_sample(sample, im_cost, tol, patchsize):
    min_cost = np.min(im_cost)
    y, x = np.where(im_cost < min_cost * (1 + tol))

    if len(y) == 0:
        # print("No valid patches found, falling back to minimum cost patch.")
        y, x = np.where(im_cost == min_cost)

    random_index = np.random.randint(len(y))
    random_y, random_x = y[random_index], x[random_index]
    patch = sample[random_y:random_y + patchsize, random_x:random_x + patchsize, :]

    return patch

def cut(errpatch):
    h, w = errpatch.shape
    cost = np.full((h, w), np.inf)
    path = np.zeros((h, w), dtype=np.int32)

    cost[:, 0] = errpatch[:, 0]
    for x in range(1, w):
        for y in range(h):
            min_cost = cost[max(y - 1, 0):min(y + 2, h), x - 1].min()
            path[y, x] = np.argmin(cost[max(y - 1, 0):min(y + 2, h), x - 1]) + max(y - 1, 0)
            cost[y, x] = errpatch[y, x] + min_cost

    seam_mask = np.zeros_like(errpatch, dtype=bool)
    y = np.argmin(cost[:, -1])
    for x in range(w - 1, -1, -1):
        seam_mask[:y, x] = True
        y = path[y, x]

    return seam_mask

def ssd_patch(sample, mask, template, x_range, y_range):
    ssd = np.zeros((sample.shape[0], sample.shape[1]), dtype=np.float32)
    for channel in range(sample.shape[2]):
        ssd += convolve2d(
            sample[..., channel]**2, mask[..., channel], mode="same"
        ) - 2 * convolve2d(
            sample[..., channel], mask[..., channel] * template[..., channel], mode="same"
        ) + np.sum((mask[..., channel] * template[..., channel])**2)

    ssd = ssd[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    return ssd

def ssd_patch_intensity(sample, mask, template, x_range, y_range, alpha, target_patch):
    sample_gray = cv2.cvtColor(sample, cv2.COLOR_BGR2GRAY).astype(np.float32)
    target_patch = target_patch.astype(np.float32)

    sample_squared = np.sum(sample ** 2, axis=2)
    mask_sum = np.sum(mask, axis=2)
    template_masked = np.sum(mask * template, axis=2)

    conv_sample_squared = convolve2d(sample_squared, mask_sum, mode="same")
    conv_sample_template = convolve2d(sample_gray, template_masked, mode="same")

    ssd_overlap = conv_sample_squared[y_range[0]:y_range[1], x_range[0]:x_range[1]] \
                  - 2 * conv_sample_template[y_range[0]:y_range[1], x_range[0]:x_range[1]] \
                  + np.sum(template_masked ** 2)

    target_squared = target_patch ** 2
    conv_sample_target = convolve2d(sample_gray, target_patch, mode="same")
    ssd_transfer = conv_sample_squared[y_range[0]:y_range[1], x_range[0]:x_range[1]] \
                   - 2 * conv_sample_target[y_range[0]:y_range[1], x_range[0]:x_range[1]] \
                   + np.sum(target_squared)

    ssd_overlap = (ssd_overlap - ssd_overlap.min()) / (ssd_overlap.max() - ssd_overlap.min() + 1e-5)
    ssd_transfer = (ssd_transfer - ssd_transfer.min()) / (ssd_transfer.max() - ssd_transfer.min() + 1e-5)

    target_weight = 200.0
    im_cost = alpha * ssd_overlap + (1 - alpha) * (target_weight * ssd_transfer)

    return im_cost

In [110]:
def quilt_random(sample, out_size, patch_size):
    sample = sample.astype(np.float32) / 255.0
    out_height, out_width = out_size
    im_out = np.zeros((out_height, out_width, sample.shape[2]), dtype=np.float32)

    x_range = [patch_size // 2, sample.shape[1] - patch_size // 2]
    y_range = [patch_size // 2, sample.shape[0] - patch_size // 2]

    for i in range(0, out_height - patch_size + 1, patch_size):
        for j in range(0, out_width - patch_size + 1, patch_size):
            patch = find_random_patch(sample, patch_size, x_range, y_range)
            im_out[i:i + patch_size, j:j + patch_size, :] = patch

    return (im_out * 255).astype(np.uint8)

In [111]:
def quilt_simple(sample, out_size, patch_size, overlap, tol):
    sample = sample.astype(np.float32) / 255.0
    out_height, out_width = out_size
    im_out = np.zeros((out_height, out_width, sample.shape[2]), dtype=np.float32)

    x_range = [patch_size // 2, sample.shape[1] - patch_size // 2]
    y_range = [patch_size // 2, sample.shape[0] - patch_size // 2]

    mask = np.zeros((patch_size, patch_size, sample.shape[2]), dtype=np.float32)
    mask[:overlap, :, :] = 1

    for i in range(0, out_height - patch_size + 1, patch_size - overlap):
        for j in range(0, out_width - patch_size + 1, patch_size - overlap):
            template = im_out[i:i + patch_size, j:j + patch_size, :]
            im_cost = ssd_patch(sample, mask, template, x_range, y_range)
            patch = choose_sample(sample, im_cost, tol, patch_size)
            im_out[i:i + patch_size, j:j + patch_size, :] = patch

    return (im_out * 255).astype(np.uint8)


In [112]:
def quilt_cut(sample, out_size, patch_size, overlap, tol):
    sample = sample.astype(np.float32) / 255.0
    out_height, out_width = out_size
    im_out = np.zeros((out_height, out_width, sample.shape[2]), dtype=np.float32)

    x_range = [patch_size // 2, sample.shape[1] - patch_size // 2]
    y_range = [patch_size // 2, sample.shape[0] - patch_size // 2]

    im_out[:patch_size, :patch_size, :] = find_random_patch(sample, patch_size, x_range, y_range)

    for i in range(0, out_height - patch_size + 1, patch_size - overlap):
        for j in range(0, out_width - patch_size + 1, patch_size - overlap):
            mask = np.zeros((patch_size, patch_size, sample.shape[2]), dtype=np.float32)

            if i > 0:
                mask[:overlap, :, :] = 1
            if j > 0:
                mask[:, :overlap, :] = 1

            template = im_out[i:i + patch_size, j:j + patch_size, :]
            im_cost = ssd_patch(sample, mask, template, x_range, y_range)
            patch = choose_sample(sample, im_cost, tol, patch_size)

            if i > 0:
                horizontal_errpath = np.sum((patch[:overlap, :, :] - template[:overlap, :, :]) ** 2, axis=2)
                horizontal_seam_mask = cut(horizontal_errpath)

                horizontal_seam_mask_expanded = horizontal_seam_mask[:, :, np.newaxis]
                patch[:overlap, :, :] = horizontal_seam_mask_expanded * patch[:overlap, :, :] + \
                                        (1 - horizontal_seam_mask_expanded) * template[:overlap, :, :]

            if j > 0:
                vertical_errpath = np.sum((patch[:, :overlap, :] - template[:, :overlap, :]) ** 2, axis=2)
                vertical_seam_mask = cut(vertical_errpath.T).T

                vertical_seam_mask_expanded = vertical_seam_mask[:, :, np.newaxis]
                patch[:, :overlap, :] = vertical_seam_mask_expanded * patch[:, :overlap, :] + \
                                        (1 - vertical_seam_mask_expanded) * template[:, :overlap, :]

            im_out[i:i + patch_size, j:j + patch_size, :] = patch

    return (im_out * 255).astype(np.uint8)

In [113]:
def texture_transfer(sample, target, patch_size, overlap, tol, alpha):
    sample = sample.astype(np.float32) / 255.0
    target = cv2.cvtColor(target.astype(np.float32) / 255.0, cv2.COLOR_BGR2GRAY)

    out_height, out_width = target.shape
    im_out = np.zeros((out_height, out_width, sample.shape[2]), dtype=np.float32)

    x_range = [patch_size // 2, sample.shape[1] - patch_size // 2]
    y_range = [patch_size // 2, sample.shape[0] - patch_size // 2]

    im_out[:patch_size, :patch_size, :] = find_random_patch(sample, patch_size, x_range, y_range)

    for i in range(0, out_height - patch_size + 1, patch_size - overlap):
        for j in range(0, out_width - patch_size + 1, patch_size - overlap):
            mask = np.zeros((patch_size, patch_size, sample.shape[2]), dtype=np.float32)

            if i > 0:
                mask[:overlap, :, :] = 1
            if j > 0:
                mask[:, :overlap, :] = 1

            template = im_out[i:i + patch_size, j:j + patch_size, :]
            target_patch = target[i:i + patch_size, j:j + patch_size]

            im_cost = ssd_patch_intensity(
                sample, mask, template, x_range, y_range, alpha, target_patch
            )
            patch = choose_sample(sample, im_cost, tol, patch_size)

            if i > 0:
                horizontal_errpath = np.sum((patch[:overlap, :, :] - template[:overlap, :, :]) ** 2, axis=2)
                horizontal_seam_mask = cut(horizontal_errpath)

                horizontal_seam_mask_expanded = horizontal_seam_mask[:, :, np.newaxis]
                patch[:overlap, :, :] = horizontal_seam_mask_expanded * patch[:overlap, :, :] + \
                                        (1 - horizontal_seam_mask_expanded) * template[:overlap, :, :]

            if j > 0:
                vertical_errpath = np.sum((patch[:, :overlap, :] - template[:, :overlap, :]) ** 2, axis=2)
                vertical_seam_mask = cut(vertical_errpath.T).T

                vertical_seam_mask_expanded = vertical_seam_mask[:, :, np.newaxis]
                patch[:, :overlap, :] = vertical_seam_mask_expanded * patch[:, :overlap, :] + \
                                        (1 - vertical_seam_mask_expanded) * template[:, :overlap, :]

            im_out[i:i + patch_size, j:j + patch_size, :] = patch

    return (im_out * 255).astype(np.uint8)

In [114]:
def texture_transfer_iterative(sample, target, patch_size, overlap, tol, alpha, iterations=3):
    sample = sample.astype(np.float32) / 255.0
    target = cv2.cvtColor(target.astype(np.float32) / 255.0, cv2.COLOR_BGR2GRAY)

    out_height, out_width = target.shape
    im_out = np.zeros((out_height, out_width, sample.shape[2]), dtype=np.float32)

    x_range = [patch_size // 2, sample.shape[1] - patch_size // 2]
    y_range = [patch_size // 2, sample.shape[0] - patch_size // 2]

    im_out[:patch_size, :patch_size, :] = find_random_patch(sample, patch_size, x_range, y_range)

    for iteration in range(iterations):
        print(f"Iteration {iteration + 1}/{iterations}")

        for i in range(0, out_height - patch_size + 1, patch_size - overlap):
            for j in range(0, out_width - patch_size + 1, patch_size - overlap):
                mask = np.zeros((patch_size, patch_size, sample.shape[2]), dtype=np.float32)

                if i > 0:
                    mask[:overlap, :, :] = 1
                if j > 0:
                    mask[:, :overlap, :] = 1

                template = im_out[i:i + patch_size, j:j + patch_size, :]
                target_patch = target[i:i + patch_size, j:j + patch_size]

                im_cost = ssd_patch_intensity(
                    sample, mask, template, x_range, y_range, alpha, target_patch
                )

                patch = choose_sample(sample, im_cost, tol, patch_size)

                if i > 0:
                    horizontal_errpath = np.sum((patch[:overlap, :, :] - template[:overlap, :, :]) ** 2, axis=2)
                    horizontal_seam_mask = cut(horizontal_errpath)
                    horizontal_seam_mask_expanded = horizontal_seam_mask[:, :, np.newaxis]
                    patch[:overlap, :, :] = horizontal_seam_mask_expanded * patch[:overlap, :, :] + \
                                            (1 - horizontal_seam_mask_expanded) * template[:overlap, :, :]

                if j > 0:
                    vertical_errpath = np.sum((patch[:, :overlap, :] - template[:, :overlap, :]) ** 2, axis=2)
                    vertical_seam_mask = cut(vertical_errpath.T).T
                    vertical_seam_mask_expanded = vertical_seam_mask[:, :, np.newaxis]
                    patch[:, :overlap, :] = vertical_seam_mask_expanded * patch[:, :overlap, :] + \
                                            (1 - vertical_seam_mask_expanded) * template[:, :overlap, :]

                im_out[i:i + patch_size, j:j + patch_size, :] = patch

    return (im_out * 255).astype(np.uint8)

In [ ]:
sample = cv2.imread('/content/drive/MyDrive/CS180_Final_Project/samples/bricks_small.jpg')
target = cv2.imread('/content/drive/MyDrive/CS180_Final_Project/samples/obama.jpg')

# Quilt Random
random_result = quilt_random(sample, (512, 512), 31)
cv2.imwrite("random_result.png", random_result)
print("random_result done")

# Quilt Simple
simple_result = quilt_simple(sample, (512, 512), 31, 4, 1)
cv2.imwrite("simple_result.png", simple_result)
print("simple_result done")

# Quilt Cut
cut_result = quilt_cut(sample, (512, 512), 31, 4, 3)
cv2.imwrite("cut_result.png", cut_result)
print("cut_result done")

# Texture Transfer
transfer_result = texture_transfer(sample, target, 41, 4, 1, 0.02)
cv2.imwrite("transfer_result.png", transfer_result)
print("transfer_result done")

# Texture Transfer with iterative refinement
transfer_iterative_result = texture_transfer_iterative(sample, target, 41, 4, 1, 0.1, iterations=4)
cv2.imwrite("transfer_result_iterative.png", transfer_iterative_result)
print("transfer_result_iterative done")
